In [2]:
using ITensors
using JLD2
using Plots
using CurveFit
using LaTeXStrings

In [3]:
function DMRG2(N; θ)
    sites = siteinds("S=1", N)

    os = OpSum()
    for j=1:N-1
        os += cos(θ)*0.5,"S+",j,"S-",j+1
        os += cos(θ)*0.5,"S-",j,"S+",j+1
        os += cos(θ),"Sz",j,"Sz",j+1

        os += sin(θ)*0.25,"S+",j,"S-",j+1,"S+",j,"S-",j+1
        os += sin(θ)*0.25,"S+",j,"S-",j+1,"S-",j,"S+",j+1
        os += sin(θ)*0.5,"S+",j,"S-",j+1,"Sz",j,"Sz",j+1

        os += sin(θ)*0.25,"S-",j,"S+",j+1,"S+",j,"S-",j+1
        os += sin(θ)*0.25,"S-",j,"S+",j+1,"S-",j,"S+",j+1
        os += sin(θ)*0.5,"S-",j,"S+",j+1,"Sz",j,"Sz",j+1

        os += sin(θ)*0.5,"Sz",j,"Sz",j+1,"S+",j,"S-",j+1
        os += sin(θ)*0.5,"Sz",j,"Sz",j+1,"S-",j,"S+",j+1
        os += sin(θ),"Sz",j,"Sz",j+1,"Sz",j,"Sz",j+1
    end
    MPH = MPO(os,sites)
    ψ = randomMPS(sites, 20)
    
    sweeps = Sweeps(30)
    setmaxdim!(sweeps, 5,5,10,10,10,25,25,25,40)
    setcutoff!(sweeps, 1E-10)
    setnoise!(sweeps, 1E-8, 1E-9, 0.0)
    
    E0, ψ0 = dmrg(MPH, ψ, sweeps, outputlevel = 0)
    
    return ψ0
end

DMRG2 (generic function with 1 method)

In [4]:
function entropies(ψ::MPS)
    L = collect(2:2:length(ψ))
    Ent = Vector{Float64}()
    
    for b in L
        orthogonalize!(ψ, b)
        U,S,V = svd(ψ[b], b > 1 ? (linkind(ψ, b-1), siteind(ψ,b)) : (siteind(ψ,b))) 
        SvN = 0.0
        #normalize!(S)
        for k = 1:dim(S, 1)
            p = S[k,k]^2
            SvN -= p*log(p)
        end
        push!(Ent, SvN)
    end

    return Ent
end

entropies (generic function with 1 method)

In [5]:
ψ = DMRG2(100, θ = -π/4)
S = entropies(ψ)
ψ = DMRG2(100, θ = atan(1/3))
S_AKLT = entropies(ψ)
save_object("S", S)
save_object("S_AKLT", S_AKLT)